In [1]:
import os
import urllib.request
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback

In [2]:
data_url = "https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/cats_and_dogs.zip"
urllib.request.urlretrieve(data_url, "cats_and_dogs.zip")
local_file = "cats_and_dogs.zip"
zip_ref = zipfile.ZipFile(local_file, "r")
zip_ref.extractall("data/")
zip_ref.close()

In [3]:
BASE_DIR = "/content/data/cats_and_dogs_filtered"
train_dir = os.path.join(BASE_DIR, "train")
validation_dir = os.path.join(BASE_DIR, "validation")

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
)

validation_datagen = ImageDataGenerator(
    rescale=1/255
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [6]:
class super_callback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.80 and logs.get('val_accuracy') > 0.80:
            self.model.stop_training = True

In [7]:
super_cb = super_callback()

In [8]:
model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Conv2D(
                16, (3, 3), activation="relu", input_shape=(150, 150, 3)
            ),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ]
    )

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(
        train_generator,
        epochs=50,
        verbose=0,
        validation_data=validation_generator,
        callbacks=[super_cb]
)

In [11]:
model.evaluate(train_generator)

100/100 [==============================] - 14s 135ms/step - loss: 0.3804 - accuracy: 0.8315


[0.380431592464447, 0.8314999938011169]

In [12]:
model.evaluate(validation_generator)

50/50 [==============================] - 3s 54ms/step - loss: 0.4387 - accuracy: 0.8000


[0.438650518655777, 0.800000011920929]